<a href="https://colab.research.google.com/github/Sounakray2003/Asmadiya-tech/blob/main/KYC_Classification_Model_Yolo11n_cls_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install -q ultralytics wandb gradio tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.7 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import cv2
import time
import random
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import gradio as gr
import wandb
from ultralytics import YOLO

# Uninstall and reinstall protobuf to avoid conflicts
!pip uninstall -y protobuf
!pip install protobuf==3.20.0

import tensorflow as tf

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-spanner 3.57.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-language 2.17.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 w

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set up WandB
wandb.login()
wandb.init(
    project="document_classification",
    name="yolov12_classification",
    config={
        "epochs": 60,
        "batch_size": 16,
        "img_size": 224,
        "patience": 10,
        "lr0": 0.0001
    }
)
config = wandb.config

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sounakray496 (sounakray496-asmadiya-technologies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Prepare dataset from specified paths
base_path = '/content/drive/MyDrive/Datasets'
aadhar_folder = os.path.join(base_path, 'aadhar_images')
passport_folder = os.path.join(base_path, 'passport_images')
pan_test_folder = os.path.join(base_path, 'pan_images', 'test')

In [ ]:
# Collect up to 204 images from each folder
aadhar_images = [os.path.join(aadhar_folder, f) for f in os.listdir(aadhar_folder) if f.endswith('.png')]
aadhar_images = aadhar_images[:204]  # Limit to 204
aadhar_labels = ['Aadhaar'] * len(aadhar_images)
passport_images = [os.path.join(passport_folder, f) for f in os.listdir(passport_folder) if f.endswith('.png')]
passport_images = random.sample(passport_images, min(204, len(passport_images)))  # Limit to 204
passport_labels = ['Passport'] * len(passport_images)
pan_images = [os.path.join(pan_test_folder, f) for f in os.listdir(pan_test_folder) if f.endswith('.png')]
pan_images = pan_images[:204]  # Limit to 204
pan_labels = ['PAN'] * len(pan_images)

In [ ]:
all_images = aadhar_images + passport_images + pan_images
all_labels = aadhar_labels + passport_labels + pan_labels
df = pd.DataFrame({'image_path': all_images, 'label': all_labels})
wandb.log({'Aadhaar_count': len(aadhar_images), 'Passport_count': len(passport_images), 'PAN_count': len(pan_images)})

In [ ]:
# Split dataset (70% train, 15% val, 15% test)
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

In [ ]:
# Create directory structure and copy images to specified paths
for split, split_df in [('train', train_df), ('val', val_df), ('test', test_df)]:
    os.makedirs(f'/content/{split}', exist_ok=True)
    os.makedirs(f'/content/{split}/Aadhaar', exist_ok=True)
    os.makedirs(f'/content/{split}/PAN', exist_ok=True)
    os.makedirs(f'/content/{split}/Passport', exist_ok=True)
    for _, row in split_df.iterrows():
        dest_folder = f'/content/{split}/{row["label"]}'
        shutil.copy(row['image_path'], dest_folder)
    # Verify population
    aadhar_count = len(os.listdir(f'/content/{split}/Aadhaar'))
    pan_count = len(os.listdir(f'/content/{split}/PAN'))
    passport_count = len(os.listdir(f'/content/{split}/Passport'))
    total_count = aadhar_count + pan_count + passport_count
    print(f"{split} at /content/{split} populated with {total_count} images: Aadhaar={aadhar_count}, PAN={pan_count}, Passport={passport_count}")

train at /content/train populated with 428 images: Aadhaar=143, PAN=143, Passport=142
val at /content/val populated with 92 images: Aadhaar=30, PAN=31, Passport=31
test at /content/test populated with 92 images: Aadhaar=31, PAN=30, Passport=31


In [ ]:
# Create and overwrite dataset.yaml at specified path
yaml_content = """
train: /content/train
val: /content/val
test: /content/test
nc: 3  # number of classes
names: ['Aadhaar', 'PAN', 'Passport']  # class names
"""
with open('/content/dataset.yaml', 'w') as f:
    f.write(yaml_content)
print("dataset.yaml created/overwritten at /content/dataset.yaml with content:", yaml_content)

dataset.yaml created/overwritten at /content/dataset.yaml with content: 
train: /content/train
val: /content/val
test: /content/test
nc: 3  # number of classes
names: ['Aadhaar', 'PAN', 'Passport']  # class names



In [ ]:
# Load YOLOv12 model - Updated to YOLOv12
# Try different possible model names for YOLO12 classification
try:
    model = YOLO('yolo12n-cls.pt')  # YOLO12 nano classification model
    print("Loaded yolo12n-cls.pt successfully")
except:
    try:
        model = YOLO('yolo12s-cls.pt')  # YOLO12 small classification model
        print("Loaded yolo12s-cls.pt successfully")
    except:
        try:
            model = YOLO('yolov12n-cls.pt')  # Alternative naming
            print("Loaded yolov12n-cls.pt successfully")
        except:
            # Fallback to YOLO11 if YOLO12 classification models aren't available yet
            model = YOLO('yolo11n-cls.pt')
            print("YOLO12 classification models not found, using YOLO11n-cls.pt as fallback")

# Train the model with verbose output and callbacks
print("Starting training with YOLOv12 and data from:", '/content/dataset.yaml')

# Custom callback to print accuracy after each epoch
def on_train_epoch_end(trainer):
    """Callback function to print metrics after each epoch"""
    if trainer.epoch >= 0:  # Start from epoch 0
        metrics = trainer.validator.metrics if hasattr(trainer, 'validator') and trainer.validator else None
        if metrics:
            # For classification, the main metrics are top1 and top5 accuracy
            top1_acc = getattr(metrics, 'top1', None)
            top5_acc = getattr(metrics, 'top5', None)

            print(f"\n=== EPOCH {trainer.epoch + 1} RESULTS ===")
            if top1_acc is not None:
                print(f"Top-1 Accuracy: {top1_acc:.4f}")
                # Log to wandb as well
                wandb.log({
                    'epoch': trainer.epoch + 1,
                    'train_top1_accuracy': top1_acc,
                    'epoch_top1_acc': top1_acc
                })
            if top5_acc is not None:
                print(f"Top-5 Accuracy: {top5_acc:.4f}")
                wandb.log({
                    'epoch': trainer.epoch + 1,
                    'train_top5_accuracy': top5_acc,
                    'epoch_top5_acc': top5_acc
                })

            # Print loss information if available
            if hasattr(trainer, 'loss_items') and trainer.loss_items is not None:
                print(f"Training Loss: {trainer.loss_items}")

            print("=" * 30)

# Add the callback to the model
model.add_callback('on_train_epoch_end', on_train_epoch_end)

YOLO12 classification models not found, using YOLO11n-cls.pt as fallback
Starting training with YOLOv12 and data from: /content/dataset.yaml


In [ ]:
# Train with verbose output
results = model.train(
    data='/content/',  # Use the root directory containing train, val, and test folders
    imgsz=config.img_size,
    epochs=config.epochs,
    batch=config.batch_size,
    patience=config.patience,
    lr0=config.lr0,
    val=True,
    project='runs/train',
    name='yolov12_exp',
    exist_ok=True,
    verbose=True,  # Enable verbose output
    plots=True,    # Generate training plots
    save_period=10  # Save model every 10 epochs
)

Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov12_exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3224.6±743.3 MB/s, size: 303.6 KB)


train: Scanning /content/train... 428 images, 0 corrupt: 100%|██████████| 428/428 [00:00<00:00, 2372.37it/s]

train: New cache created: /content/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1459.5±1238.5 MB/s, size: 262.7 KB)


...

               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 21.54it/s]

                   all      0.967          1


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

19 epochs completed in 0.039 hours.
Optimizer stripped from runs/train/yolov12_exp/weights/last.pt, 3.2MB
Optimizer stripped from runs/train/yolov12_exp/weights/best.pt, 3.2MB

Validating runs/train/yolov12_exp/weights/best.pt...
Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,529,867 parameters, 0 gradients, 3.2 GFLOPs
train: /content/train... found 428 images in 3 classes ✅ 
val: /content/val... found 92 images in 3 classes ✅ 
test: /content/test... found 92 images in 3 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 14.64it/s]


                   all      0.989          1
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/train/yolov12_exp


In [ ]:
# Evaluate the model
metrics = model.val()
test_accuracy = metrics.top1  # Using top-1 accuracy for classification
wandb.log({'test_accuracy': test_accuracy})

Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,529,867 parameters, 0 gradients, 3.2 GFLOPs
train: /content/train... found 428 images in 3 classes ✅ 
val: /content/val... found 92 images in 3 classes ✅ 
test: /content/test... found 92 images in 3 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1834.4±1077.3 MB/s, size: 262.7 KB)


val: Scanning /content/val... 92 images, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:01<00:00,  5.30it/s]


                   all      0.989          1
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/train/yolov12_exp


In [ ]:
# Predictions for test set
predictions = model.predict(test_df['image_path'].tolist(), imgsz=config.img_size)
predicted_classes = np.array([x.probs.top1 for x in predictions])  # Convert to numpy array
true_classes = test_df['label'].map({'Aadhaar': 0, 'PAN': 1, 'Passport': 2}).values

print(f"True classes: {np.unique(true_classes, return_counts=True)}")
print(f"Predicted classes: {np.unique(predicted_classes, return_counts=True)}")
print(f"True classes array: {true_classes}")
print(f"Predicted classes array: {predicted_classes}")


0: 224x224 PAN 1.00, Passport 0.00, Aadhaar 0.00, 0.3ms
1: 224x224 PAN 1.00, Aadhaar 0.00, Passport 0.00, 0.3ms
2: 224x224 PAN 0.98, Aadhaar 0.02, Passport 0.01, 0.3ms
3: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
4: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
5: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
6: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
7: 224x224 Passport 1.00, Aadhaar 0.00, PAN 0.00, 0.3ms
8: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
9: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
10: 224x224 Aadhaar 0.96, PAN 0.04, Passport 0.00, 0.3ms
11: 224x224 PAN 1.00, Passport 0.00, Aadhaar 0.00, 0.3ms
12: 224x224 Passport 1.00, Aadhaar 0.00, PAN 0.00, 0.3ms
13: 224x224 Passport 1.00, Aadhaar 0.00, PAN 0.00, 0.3ms
14: 224x224 PAN 1.00, Passport 0.00, Aadhaar 0.00, 0.3ms
15: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
16: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
17: 224x224 Passport 1.00, Aadhaar 0.00,

In [ ]:
# Calculate accuracy for each class
for i, label in enumerate(['Aadhaar', 'PAN', 'Passport']):
    mask = true_classes == i
    if np.sum(mask) > 0:
        matches = predicted_classes[mask] == i
        accuracy = np.mean(matches)
        print(f"Class {label} accuracy: {accuracy:.4f}")
        wandb.log({f'{label}_accuracy': accuracy})

Class Aadhaar accuracy: 1.0000
Class PAN accuracy: 0.9667
Class Passport accuracy: 1.0000


In [ ]:
# Calculate overall accuracy
overall_accuracy = np.mean(predicted_classes == true_classes)
print(f"Overall Test Accuracy: {overall_accuracy:.4f}")
wandb.log({'overall_test_accuracy': overall_accuracy})

Overall Test Accuracy: 0.9891


In [ ]:
# Save the model with YOLOv12 naming
model.save('/content/drive/MyDrive/document_classifier_yolo12.pt')
artifact = wandb.Artifact('document_classifier_yolo12_model', type='model')
artifact.add_file('/content/drive/MyDrive/document_classifier_yolo12.pt')
wandb.log_artifact(artifact)

<Artifact document_classifier_yolo12_model>

In [ ]:
# Load the trained model (use the best weights from training)
try:
    # Try to load the best model from training results
    model_path = 'runs/train/yolov12_exp/weights/best.pt'
    if os.path.exists(model_path):
        model = YOLO(model_path)
        print(f"Loaded best trained model from: {model_path}")
    else:
        # Fallback to saved model
        model = YOLO('/content/drive/MyDrive/document_classifier_yolo12.pt')
        print("Loaded model from Drive")
except Exception as e:
    print(f"Error loading model: {e}")
    # Last resort - load the original trained model object
    print("Using the model object from training")

Loaded best trained model from: runs/train/yolov12_exp/weights/best.pt


In [ ]:
def is_blurry(image_np, threshold=150):
    if len(image_np.shape) == 3:
        gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    else:
        gray = image_np
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    wandb.log({'laplacian_variance': laplacian_var})
    return laplacian_var < threshold

output_dir = '/content/drive/MyDrive/Output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for label in ['Aadhaar', 'PAN', 'Passport']:
    folder = os.path.join(output_dir, label)
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
def classify_document(selected_type, image):
    blur_result = is_blurry(image)

    # Proper preprocessing for YOLO model
    # Convert RGB to BGR if needed (YOLO expects BGR)
    if len(image.shape) == 3 and image.shape[2] == 3:
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    else:
        image_bgr = image

    # Let YOLO handle the preprocessing instead of manual resizing
    predictions = model.predict(image_bgr, imgsz=config.img_size, verbose=False)[0]

    # Get prediction results
    predicted_class_idx = predictions.probs.top1
    predicted_label = ['Aadhaar', 'PAN', 'Passport'][predicted_class_idx]
    confidence = predictions.probs.top1conf.item()

    # Get all class probabilities for debugging
    class_probs = predictions.probs.data.cpu().numpy() if hasattr(predictions.probs.data, 'cpu') else predictions.probs.data

    print(f"Class probabilities - Aadhaar: {class_probs[0]:.4f}, PAN: {class_probs[1]:.4f}, Passport: {class_probs[2]:.4f}")
    print(f"Selected: {selected_type}, Predicted: {predicted_label}, Confidence: {confidence:.4f}, Blurry: {blur_result}")

    wandb.log({
        'selected_type': selected_type,
        'predicted_label': predicted_label,
        'confidence': confidence,
        'blurry': blur_result,
        'aadhaar_prob': float(class_probs[0]),
        'pan_prob': float(class_probs[1]),
        'passport_prob': float(class_probs[2])
    })

    if blur_result:
        return "Please upload a clearer image."

    # Lower the confidence threshold temporarily for debugging
    confidence_threshold = 0.3  # Reduced from 0.8 for testing

    if predicted_label == selected_type and confidence >= confidence_threshold:
        timestamp = int(time.time())
        filename = f"{selected_type}_{timestamp}.png"
        save_path = os.path.join(output_dir, selected_type, filename)
        image_bgr_save = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, image_bgr_save)
        return f"Document accepted. Confidence: {confidence:.3f}"
    else:
        if predicted_label == selected_type:
            return f"Document type correct but confidence too low ({confidence:.3f}). Please try a clearer image."
        else:
            if selected_type == 'Passport':
                return f"Please upload a valid Indian passport. Detected as {predicted_label} with {confidence:.3f} confidence."
            else:
                return f"Please upload a valid {selected_type} document. Detected as {predicted_label} with {confidence:.3f} confidence."


In [ ]:
iface = gr.Interface(
    fn=classify_document,
    inputs=[
        gr.Dropdown(choices=['Aadhaar', 'PAN', 'Passport'], label="Select Document Type"),
        gr.Image(type="numpy")
    ],
    outputs="text",
    title="Document Classifier - YOLOv12"
)

In [ ]:
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://991209b8919a3ec1a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Class probabilities - Aadhaar: 0.0011, PAN: 0.0035, Passport: 0.9954
Selected: Passport, Predicted: Passport, Confidence: 0.9954, Blurry: False
Class probabilities - Aadhaar: 0.0000, PAN: 0.0000, Passport: 1.0000
Selected: Passport, Predicted: Passport, Confidence: 1.0000, Blurry: False
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://991209b8919a3ec1a3.gradio.live


In [ ]:
wandb.finish()

In [ ]:
# Install required packages
!pip install -q ultralytics wandb gradio tensorflow

# Import libraries
import os
import pandas as pd
import numpy as np
import cv2
import time
import random
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import gradio as gr
import wandb
from ultralytics import YOLO

# Uninstall and reinstall protobuf to avoid conflicts
!pip uninstall -y protobuf
!pip install protobuf==3.20.0

import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

# Set up WandB
wandb.login()
wandb.init(
    project="document_classification",
    name="yolov12_classification",
    config={
        "epochs": 60,
        "batch_size": 16,
        "img_size": 224,
        "patience": 10,
        "lr0": 0.0001
    }
)
config = wandb.config

# Prepare dataset from specified paths
base_path = '/content/drive/MyDrive/Datasets'
aadhar_folder = os.path.join(base_path, 'aadhar_images')
passport_folder = os.path.join(base_path, 'passport_images')
pan_test_folder = os.path.join(base_path, 'pan_images', 'test')

# Collect up to 204 images from each folder
aadhar_images = [os.path.join(aadhar_folder, f) for f in os.listdir(aadhar_folder) if f.endswith('.png')]
aadhar_images = aadhar_images[:204]  # Limit to 204
aadhar_labels = ['Aadhaar'] * len(aadhar_images)
passport_images = [os.path.join(passport_folder, f) for f in os.listdir(passport_folder) if f.endswith('.png')]
passport_images = random.sample(passport_images, min(204, len(passport_images)))  # Limit to 204
passport_labels = ['Passport'] * len(passport_images)
pan_images = [os.path.join(pan_test_folder, f) for f in os.listdir(pan_test_folder) if f.endswith('.png')]
pan_images = pan_images[:204]  # Limit to 204
pan_labels = ['PAN'] * len(pan_images)

all_images = aadhar_images + passport_images + pan_images
all_labels = aadhar_labels + passport_labels + pan_labels
df = pd.DataFrame({'image_path': all_images, 'label': all_labels})
wandb.log({'Aadhaar_count': len(aadhar_images), 'Passport_count': len(passport_images), 'PAN_count': len(pan_images)})

# Split dataset (70% train, 15% val, 15% test)
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Create directory structure and copy images to specified paths
for split, split_df in [('train', train_df), ('val', val_df), ('test', test_df)]:
    os.makedirs(f'/content/{split}', exist_ok=True)
    os.makedirs(f'/content/{split}/Aadhaar', exist_ok=True)
    os.makedirs(f'/content/{split}/PAN', exist_ok=True)
    os.makedirs(f'/content/{split}/Passport', exist_ok=True)
    for _, row in split_df.iterrows():
        dest_folder = f'/content/{split}/{row["label"]}'
        shutil.copy(row['image_path'], dest_folder)
    # Verify population
    aadhar_count = len(os.listdir(f'/content/{split}/Aadhaar'))
    pan_count = len(os.listdir(f'/content/{split}/PAN'))
    passport_count = len(os.listdir(f'/content/{split}/Passport'))
    total_count = aadhar_count + pan_count + passport_count
    print(f"{split} at /content/{split} populated with {total_count} images: Aadhaar={aadhar_count}, PAN={pan_count}, Passport={passport_count}")

# Create and overwrite dataset.yaml at specified path
yaml_content = """
train: /content/train
val: /content/val
test: /content/test
nc: 3  # number of classes
names: ['Aadhaar', 'PAN', 'Passport']  # class names
"""
with open('/content/dataset.yaml', 'w') as f:
    f.write(yaml_content)
print("dataset.yaml created/overwritten at /content/dataset.yaml with content:", yaml_content)

# Load YOLOv12 model - Updated to YOLOv12
# Try different possible model names for YOLO12 classification
try:
    model = YOLO('yolo12n-cls.pt')  # YOLO12 nano classification model
    print("Loaded yolo12n-cls.pt successfully")
except:
    try:
        model = YOLO('yolo12s-cls.pt')  # YOLO12 small classification model
        print("Loaded yolo12s-cls.pt successfully")
    except:
        try:
            model = YOLO('yolov12n-cls.pt')  # Alternative naming
            print("Loaded yolov12n-cls.pt successfully")
        except:
            # Fallback to YOLO11 if YOLO12 classification models aren't available yet
            model = YOLO('yolo11n-cls.pt')
            print("YOLO12 classification models not found, using YOLO11n-cls.pt as fallback")

# Train the model with verbose output and callbacks
print("Starting training with YOLOv12 and data from:", '/content/dataset.yaml')

# Custom callback to print accuracy after each epoch
def on_train_epoch_end(trainer):
    """Callback function to print metrics after each epoch"""
    if trainer.epoch >= 0:  # Start from epoch 0
        metrics = trainer.validator.metrics if hasattr(trainer, 'validator') and trainer.validator else None
        if metrics:
            # For classification, the main metrics are top1 and top5 accuracy
            top1_acc = getattr(metrics, 'top1', None)
            top5_acc = getattr(metrics, 'top5', None)

            print(f"\n=== EPOCH {trainer.epoch + 1} RESULTS ===")
            if top1_acc is not None:
                print(f"Top-1 Accuracy: {top1_acc:.4f}")
                # Log to wandb as well
                wandb.log({
                    'epoch': trainer.epoch + 1,
                    'train_top1_accuracy': top1_acc,
                    'epoch_top1_acc': top1_acc
                })
            if top5_acc is not None:
                print(f"Top-5 Accuracy: {top5_acc:.4f}")
                wandb.log({
                    'epoch': trainer.epoch + 1,
                    'train_top5_accuracy': top5_acc,
                    'epoch_top5_acc': top5_acc
                })

            # Print loss information if available
            if hasattr(trainer, 'loss_items') and trainer.loss_items is not None:
                print(f"Training Loss: {trainer.loss_items}")

            print("=" * 30)

# Add the callback to the model
model.add_callback('on_train_epoch_end', on_train_epoch_end)

# Train with verbose output
results = model.train(
    data='/content/',  # Use the root directory containing train, val, and test folders
    imgsz=config.img_size,
    epochs=config.epochs,
    batch=config.batch_size,
    patience=config.patience,
    lr0=config.lr0,
    val=True,
    project='runs/train',
    name='yolov12_exp',
    exist_ok=True,
    verbose=True,  # Enable verbose output
    plots=True,    # Generate training plots
    save_period=10  # Save model every 10 epochs
)

# Evaluate the model
metrics = model.val()
test_accuracy = metrics.top1  # Using top-1 accuracy for classification
wandb.log({'test_accuracy': test_accuracy})

# Predictions for test set
predictions = model.predict(test_df['image_path'].tolist(), imgsz=config.img_size)
predicted_classes = np.array([x.probs.top1 for x in predictions])  # Convert to numpy array
true_classes = test_df['label'].map({'Aadhaar': 0, 'PAN': 1, 'Passport': 2}).values

print(f"True classes: {np.unique(true_classes, return_counts=True)}")
print(f"Predicted classes: {np.unique(predicted_classes, return_counts=True)}")
print(f"True classes array: {true_classes}")
print(f"Predicted classes array: {predicted_classes}")

# Calculate accuracy for each class
for i, label in enumerate(['Aadhaar', 'PAN', 'Passport']):
    mask = true_classes == i
    if np.sum(mask) > 0:
        matches = predicted_classes[mask] == i
        accuracy = np.mean(matches)
        print(f"Class {label} accuracy: {accuracy:.4f}")
        wandb.log({f'{label}_accuracy': accuracy})

# Calculate overall accuracy
overall_accuracy = np.mean(predicted_classes == true_classes)
print(f"Overall Test Accuracy: {overall_accuracy:.4f}")
wandb.log({'overall_test_accuracy': overall_accuracy})

# Save the model with YOLOv12 naming
model.save('/content/drive/MyDrive/document_classifier_yolo12.pt')
artifact = wandb.Artifact('document_classifier_yolo12_model', type='model')
artifact.add_file('/content/drive/MyDrive/document_classifier_yolo12.pt')
wandb.log_artifact(artifact)

# Load the trained model (use the best weights from training)
try:
    # Try to load the best model from training results
    model_path = 'runs/train/yolov12_exp/weights/best.pt'
    if os.path.exists(model_path):
        model = YOLO(model_path)
        print(f"Loaded best trained model from: {model_path}")
    else:
        # Fallback to saved model
        model = YOLO('/content/drive/MyDrive/document_classifier_yolo12.pt')
        print("Loaded model from Drive")
except Exception as e:
    print(f"Error loading model: {e}")
    # Last resort - load the original trained model object
    print("Using the model object from training")

def is_blurry(image_np, threshold=150):
    if len(image_np.shape) == 3:
        gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    else:
        gray = image_np
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    wandb.log({'laplacian_variance': laplacian_var})
    return laplacian_var < threshold

output_dir = '/content/drive/MyDrive/Output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for label in ['Aadhaar', 'PAN', 'Passport']:
    folder = os.path.join(output_dir, label)
    if not os.path.exists(folder):
        os.makedirs(folder)

def classify_document(selected_type, image):
    blur_result = is_blurry(image)

    # Proper preprocessing for YOLO model
    # Convert RGB to BGR if needed (YOLO expects BGR)
    if len(image.shape) == 3 and image.shape[2] == 3:
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    else:
        image_bgr = image

    # Let YOLO handle the preprocessing instead of manual resizing
    predictions = model.predict(image_bgr, imgsz=config.img_size, verbose=False)[0]

    # Get prediction results
    predicted_class_idx = predictions.probs.top1
    predicted_label = ['Aadhaar', 'PAN', 'Passport'][predicted_class_idx]
    confidence = predictions.probs.top1conf.item()

    # Get all class probabilities for debugging
    class_probs = predictions.probs.data.cpu().numpy() if hasattr(predictions.probs.data, 'cpu') else predictions.probs.data

    print(f"Class probabilities - Aadhaar: {class_probs[0]:.4f}, PAN: {class_probs[1]:.4f}, Passport: {class_probs[2]:.4f}")
    print(f"Selected: {selected_type}, Predicted: {predicted_label}, Confidence: {confidence:.4f}, Blurry: {blur_result}")

    wandb.log({
        'selected_type': selected_type,
        'predicted_label': predicted_label,
        'confidence': confidence,
        'blurry': blur_result,
        'aadhaar_prob': float(class_probs[0]),
        'pan_prob': float(class_probs[1]),
        'passport_prob': float(class_probs[2])
    })

    if blur_result:
        return "Please upload a clearer image."

    # Lower the confidence threshold temporarily for debugging
    confidence_threshold = 0.3  # Reduced from 0.8 for testing

    if predicted_label == selected_type and confidence >= confidence_threshold:
        timestamp = int(time.time())
        filename = f"{selected_type}_{timestamp}.png"
        save_path = os.path.join(output_dir, selected_type, filename)
        image_bgr_save = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, image_bgr_save)
        return f"Document accepted. Confidence: {confidence:.3f}"
    else:
        if predicted_label == selected_type:
            return f"Document type correct but confidence too low ({confidence:.3f}). Please try a clearer image."
        else:
            if selected_type == 'Passport':
                return f"Please upload a valid Indian passport. Detected as {predicted_label} with {confidence:.3f} confidence."
            else:
                return f"Please upload a valid {selected_type} document. Detected as {predicted_label} with {confidence:.3f} confidence."

iface = gr.Interface(
    fn=classify_document,
    inputs=[
        gr.Dropdown(choices=['Aadhaar', 'PAN', 'Passport'], label="Select Document Type"),
        gr.Image(type="numpy")
    ],
    outputs="text",
    title="Document Classifier - YOLOv12"
)

iface.launch(debug=True)

wandb.finish()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 17.6 MB/s eta 0:00:00
Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
  Using cached protobuf-3.20.0-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.0-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-spanner 3.57.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-language 2.17.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-speech 2.33.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is inco

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Aadhaar_accuracy,▁
Aadhaar_count,▁
PAN_accuracy,▁
PAN_count,▁
Passport_accuracy,▁
Passport_count,▁
aadhaar_prob,█▁
confidence,▁█
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
epoch_top1_acc,▁███████▇██████████
epoch_top5_acc,▁██████████████████


train at /content/train populated with 468 images: Aadhaar=143, PAN=143, Passport=182
val at /content/val populated with 118 images: Aadhaar=30, PAN=31, Passport=57
test at /content/test populated with 115 images: Aadhaar=31, PAN=30, Passport=54
dataset.yaml created/overwritten at /content/dataset.yaml with content: 
train: /content/train
val: /content/val
test: /content/test
nc: 3  # number of classes
names: ['Aadhaar', 'PAN', 'Passport']  # class names

YOLO12 classification models not found, using YOLO11n-cls.pt as fallback
Starting training with YOLOv12 and data from: /content/dataset.yaml
Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/, degrees=0.0, deterministic=True, device=None, d

...

                   all          1          1
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/train/yolov12_exp
Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,529,867 parameters, 0 gradients, 3.2 GFLOPs
train: /content/train... found 468 images in 3 classes ✅ 
val: /content/val... found 118 images in 3 classes ✅ 
test: /content/test... found 115 images in 3 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2190.0±612.0 MB/s, size: 262.7 KB)


val: Scanning /content/val... 118 images, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:01<00:00,  5.03it/s]


                   all          1          1
Speed: 0.5ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/train/yolov12_exp

0: 224x224 PAN 1.00, Aadhaar 0.00, Passport 0.00, 0.3ms
1: 224x224 PAN 1.00, Aadhaar 0.00, Passport 0.00, 0.3ms
2: 224x224 PAN 0.99, Aadhaar 0.01, Passport 0.00, 0.3ms
3: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
4: 224x224 Aadhaar 0.99, PAN 0.01, Passport 0.00, 0.3ms
5: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
6: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
7: 224x224 Passport 1.00, PAN 0.00, Aadhaar 0.00, 0.3ms
8: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
9: 224x224 Aadhaar 1.00, PAN 0.00, Passport 0.00, 0.3ms
10: 224x224 PAN 0.94, Aadhaar 0.04, Passport 0.02, 0.3ms
11: 224x224 PAN 1.00, Aadhaar 0.00, Passport 0.00, 0.3ms
12: 224x224 Passport 1.00, Aadhaar 0.00, PAN 0.00, 0.3ms
13: 224x224 Passport 1.00, Aadhaar 0.00, PAN 0.00, 0.3ms
14: 224x224 PAN 1.00, Aadhaar 0.00, Passport

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://660441dd423c83b1a1.gradio.live


Aadhaar_accuracy,▁
Aadhaar_count,▁
PAN_accuracy,▁
PAN_count,▁
Passport_accuracy,▁
Passport_count,▁
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇█
epoch_top1_acc,▁██████████████████████████
epoch_top5_acc,▁██████████████████████████
overall_test_accuracy,▁
test_accuracy,▁


In [ ]:
!pip install pytesseract

In [ ]:
import re
import json
import cv2
import numpy as np
from PIL import Image
import pytesseract
import base64
import io
import gradio as gr

def extract_aadhaar_data(image):
    img_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # OCR line-wise (clean non-ASCII)
    raw_text = pytesseract.image_to_string(image, lang="eng")
    lines = [re.sub(r'[^\x00-\x7F]+', '', l).strip() for l in raw_text.split("\n") if l.strip()]

    aadhaar_number = None
    name = None
    dob = None
    gender = None

    # Aadhaar number search
    for l in lines:
        m = re.search(r"\b[2-9]{1}[0-9]{3}\s?[0-9]{4}\s?[0-9]{4}\b", l)
        if m:
            aadhaar_number = m.group(0)
            break

    # DOB & Name extraction
    for i, l in enumerate(lines):
        dob_match = re.search(r"(\d{2}[\/\-]\d{2}[\/\-]\d{4})", l)
        if dob_match:
            dob = dob_match.group(1)
            if i > 0:
                prev_line = lines[i-1]
                name = " ".join(re.findall(r"[A-Za-z]+", prev_line))
            break

    # Gender detection
    for l in lines:
        if re.search(r"\bMALE\b", l, re.IGNORECASE):
            gender = "Male"
            break
        elif re.search(r"\bFEMALE\b", l, re.IGNORECASE):
            gender = "Female"
            break

    # Face detection & crop
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(50, 50))

    cropped_face_img = None
    user_photo_base64 = None
    if len(faces) > 0:
        x, y, w, h = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)[0]
        cropped_face_img = img_bgr[y:y + h, x:x + w]
        pil_face = Image.fromarray(cv2.cvtColor(cropped_face_img, cv2.COLOR_BGR2RGB))
        buf = io.BytesIO()
        pil_face.save(buf, format="PNG")
        user_photo_base64 = base64.b64encode(buf.getvalue()).decode()

    # JSON output
    data = {
        "aadhaar_number": aadhaar_number,
        "name": name,
        "dob": dob,
        "gender": gender,
        "user_photo_base64": user_photo_base64
    }

    face_image_display = None
    if cropped_face_img is not None:
        face_image_display = cv2.cvtColor(cropped_face_img, cv2.COLOR_BGR2RGB)

    return json.dumps(data, indent=4, ensure_ascii=False), face_image_display


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Aadhaar Card Text & Photo Extractor")
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(type="pil", label="Upload Aadhaar Image")
            btn = gr.Button("Extract Data")
        with gr.Column():
            output_json = gr.Textbox(label="Extracted Data (JSON)", lines=15)
            output_face = gr.Image(type="numpy", label="Cropped User Photo")

    btn.click(fn=extract_aadhaar_data, inputs=input_img, outputs=[output_json, output_face])

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://18e18d40268cc7b728.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://18e18d40268cc7b728.gradio.live


In [ ]:
import re
import json
import cv2
import numpy as np
from PIL import Image
import pytesseract
import base64
import io
import gradio as gr

def clean_text(text):
    """Remove unwanted headers and keep only meaningful name-like text."""
    if not text:
        return None
    text = text.strip()
    ignore_patterns = [
        "INCOME TAX", "GOVT OF INDIA", "GOVERNMENT OF INDIA",
        "INCOME", "TAX", "DEPARTMENT", "Permanent Account Number"
    ]
    for pat in ignore_patterns:
        if pat.lower() in text.lower():
            return None
    return text

def extract_pan_data(image):
    img_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # OCR line-wise (clean non-ASCII)
    raw_text = pytesseract.image_to_string(image, lang="eng")
    lines = [re.sub(r'[^\x00-\x7F]+', '', l).strip() for l in raw_text.split("\n") if l.strip()]

    pan_number = None
    name = None
    father_name = None
    dob = None

    # PAN number search (ABCDE1234F)
    for l in lines:
        m = re.search(r"\b([A-Z]{5}[0-9]{4}[A-Z]{1})\b", l)
        if m:
            pan_number = m.group(1)
            break

    # DOB extraction & Name
    for i, l in enumerate(lines):
        dob_match = re.search(r"(\d{2}[\/\-]\d{2}[\/\-]\d{4})", l)
        if dob_match:
            dob = dob_match.group(1)
            if i > 0:
                father_name = clean_text(lines[i-1])
            if i > 1:
                name = clean_text(lines[i-2])
            break

    # Face detection & crop
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(50, 50))

    cropped_face_img = None
    user_photo_base64 = None
    if len(faces) > 0:
        x, y, w, h = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)[0]
        cropped_face_img = img_bgr[y:y + h, x:x + w]
        pil_face = Image.fromarray(cv2.cvtColor(cropped_face_img, cv2.COLOR_BGR2RGB))
        buf = io.BytesIO()
        pil_face.save(buf, format="PNG")
        user_photo_base64 = base64.b64encode(buf.getvalue()).decode()

    # JSON output
    data = {
        "pan_number": pan_number,
        "name": name,
        "father_name": father_name,
        "dob": dob,
        "user_photo_base64": user_photo_base64
    }

    face_image_display = None
    if cropped_face_img is not None:
        face_image_display = cv2.cvtColor(cropped_face_img, cv2.COLOR_BGR2RGB)

    return json.dumps(data, indent=4, ensure_ascii=False), face_image_display


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## PAN Card Text & Photo Extractor")
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(type="pil", label="Upload PAN Image")
            btn = gr.Button("Extract Data")
        with gr.Column():
            output_json = gr.Textbox(label="Extracted Data (JSON)", lines=15)
            output_face = gr.Image(type="numpy", label="Cropped User Photo")

    btn.click(fn=extract_pan_data, inputs=input_img, outputs=[output_json, output_face])

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e7c4ebd7fa0ecf4693.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e7c4ebd7fa0ecf4693.gradio.live


In [ ]:
import re
import pytesseract
from PIL import Image
import gradio as gr
import cv2
import numpy as np

def preprocess_image(image, method="standard"):
    """Enhance image for better OCR with multiple preprocessing methods"""
    img_array = np.array(image)
    img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    if method == "standard":
        # Basic enhancement
        denoised = cv2.fastNlMeansDenoising(gray)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        return clahe.apply(denoised)

    elif method == "sharp":
        # Sharpening for clearer text
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        sharpened = cv2.filter2D(gray, -1, kernel)
        return cv2.fastNlMeansDenoising(sharpened)

    elif method == "threshold":
        # Binary threshold for high contrast
        denoised = cv2.fastNlMeansDenoising(gray)
        _, thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thresh

    return gray

def clean_text(text, field_type="generic"):
    """Clean and normalize extracted text"""
    if not text:
        return ""

    text = text.strip().replace("\n", " ")

    if field_type == "name":
        # Remove non-alphabetic characters but keep spaces and common name characters
        cleaned = re.sub(r'[^A-Za-z\s\'-]', '', text).strip()
        # Remove extra spaces and convert to title case
        cleaned = re.sub(r'\s+', ' ', cleaned).title()
        # Handle common OCR corrections
        cleaned = fix_common_ocr_errors(cleaned)
        return cleaned
    elif field_type == "date":
        # Remove spaces and normalize separators
        return re.sub(r'[^\d/\-]', '', text)
    elif field_type == "passport_number":
        # Keep only alphanumeric characters
        return re.sub(r'[^A-Z0-9]', '', text.upper())

    return re.sub(r'\s+', ' ', text).strip()

def fix_common_ocr_errors(name):
    """Fix common OCR errors in names"""
    if not name:
        return name

    # Common OCR substitutions
    ocr_fixes = {
        # Letter confusions
        '0': 'O',  # Zero to O
        '1': 'I',  # One to I (in names)
        '5': 'S',  # Five to S
        '8': 'B',  # Eight to B
        '@': 'A',  # At symbol to A
        '3': 'E',  # Three to E
        '6': 'G',  # Six to G
        # Common word fixes
        'Fe male': 'Female',
        'Ma le': 'Male',
        'Fern': 'F',  # Common OCR error for F
        'San': '',   # Common OCR garbage
        'Fem': '',   # Common OCR garbage
    }

    result = name
    for wrong, correct in ocr_fixes.items():
        result = result.replace(wrong, correct)

    return result

def is_valid_name(name_str):
    """Enhanced name validation with better heuristics"""
    if not name_str or len(name_str) < 2:
        return False

    # Remove common OCR artifacts and check if it's reasonable
    cleaned = re.sub(r'[^A-Za-z\s\'-]', '', name_str).strip()

    if not cleaned:
        return False

    # Check for reasonable characteristics of names
    words = cleaned.split()
    if len(words) == 0:
        return False

    # Names should have reasonable length words
    valid_words = 0
    for word in words:
        if len(word) >= 2:  # At least 2 characters
            valid_words += 1
        elif len(word) == 1 and word.upper() in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
            valid_words += 1  # Single letters are OK (middle initials)

    if valid_words == 0:
        return False

    # Check against common OCR errors that aren't names
    ocr_garbage = [
        'fem', 'san', 'the', 'and', 'for', 'with', 'date', 'sex', 'place',
        'republic', 'india', 'passport', 'number', 'nationality', 'given',
        'surname', 'birth', 'issue', 'expiry', 'authority', 'type', 'code',
        'holder', 'male', 'female', 'country', 'document', 'personal'
    ]

    # Check if the entire cleaned text is just garbage
    if cleaned.lower() in ocr_garbage:
        return False

    # Check if it's mostly garbage words
    garbage_count = sum(1 for word in words if word.lower() in ocr_garbage)
    if len(words) > 1 and garbage_count >= len(words) // 2:
        return False

    # Check for patterns that suggest OCR errors
    error_patterns = [
        r'^[A-Z]{1,2}\s+[A-Z]{1,2}$',  # Very short words like "FE MA"
        r'.*\d.*',  # Names shouldn't contain numbers
        r'^[^A-Za-z]*$',  # Should contain at least some letters
    ]

    for pattern in error_patterns:
        if re.match(pattern, cleaned):
            return False

    # Additional validation: check if it looks like a real name structure
    # Names typically have 1-4 words, each 2-20 characters long
    if len(words) > 4:
        return False

    for word in words:
        if len(word) > 20:  # Unusually long word
            return False

    return True

def extract_name_candidates(text):
    """Extract potential name candidates from text using multiple strategies"""
    candidates = []

    # Strategy 1: Context-based extraction
    name_contexts = [
        # Surname patterns
        (r'(?:Surname|Family\s*Name|Last\s*Name)[\s:]*([A-Z][A-Za-z\s\'-]+?)(?=\s*(?:\n|Given|First|Christian|Forename|Nationality|Sex|Date|Place|$))', 'surname'),
        (r'(?:^|\n)([A-Z][A-Z\s]{3,25})(?=\s*\n.*(?:Given|First|Christian|Forename))', 'surname'),

        # Given names patterns
        (r'(?:Given\s*Names?|First\s*Names?|Christian\s*Names?|Forenames?)[\s:]*([A-Z][A-Za-z\s\'-]+?)(?=\s*(?:\n|Nationality|Sex|Date|Place|$))', 'given_names'),
        (r'(?:Given\s*Names?|First\s*Names?)[\s:]*\n?\s*([A-Z][A-Za-z\s\'-]+)', 'given_names'),

        # General name patterns (when context is unclear)
        (r'(?:^|\n)([A-Z][A-Z\s]{2,25})(?=\s*\n)', 'general'),
        (r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b', 'general'),
    ]

    for pattern, name_type in name_contexts:
        matches = re.finditer(pattern, text, re.MULTILINE | re.IGNORECASE)
        for match in matches:
            candidate = clean_text(match.group(1), "name")
            if candidate and is_valid_name(candidate):
                candidates.append({
                    'text': candidate,
                    'type': name_type,
                    'confidence': calculate_name_confidence(candidate, name_type, text),
                    'source': 'context'
                })

    # Strategy 2: MRZ extraction
    mrz_candidates = extract_mrz_names(text)
    candidates.extend(mrz_candidates)

    # Strategy 3: Line-by-line analysis for passport format
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line = line.strip()
        if len(line) > 3 and re.match(r'^[A-Z][A-Z\s]{2,}$', line):
            candidate = clean_text(line, "name")
            if candidate and is_valid_name(candidate):
                # Determine type based on position and context
                name_type = 'general'
                if i < len(lines) - 1:
                    next_line = lines[i + 1].lower()
                    if any(word in next_line for word in ['given', 'first', 'christian', 'forename']):
                        name_type = 'surname'
                    elif any(word in next_line for word in ['nationality', 'sex', 'date']):
                        name_type = 'given_names'

                candidates.append({
                    'text': candidate,
                    'type': name_type,
                    'confidence': calculate_name_confidence(candidate, name_type, text),
                    'source': 'line_analysis'
                })

    return candidates

def extract_mrz_names(text):
    """Extract names from MRZ format"""
    candidates = []

    # MRZ pattern for names: P<COUNTRY<SURNAME<<GIVEN_NAMES<<<
    mrz_patterns = [
        r'P<[A-Z]{3}<([A-Z<]+)<<([A-Z<]*)',
        r'P<[A-Z]{3}<([A-Z<]+)<([A-Z<]*)',
    ]

    for pattern in mrz_patterns:
        match = re.search(pattern, text)
        if match:
            surname_mrz = match.group(1).replace('<', ' ').strip()
            given_names_mrz = match.group(2).replace('<', ' ').strip()

            if surname_mrz:
                surname_clean = clean_text(surname_mrz, "name")
                if surname_clean and is_valid_name(surname_clean):
                    candidates.append({
                        'text': surname_clean,
                        'type': 'surname',
                        'confidence': 0.9,  # MRZ is usually reliable
                        'source': 'mrz'
                    })

            if given_names_mrz:
                given_clean = clean_text(given_names_mrz, "name")
                if given_clean and is_valid_name(given_clean):
                    candidates.append({
                        'text': given_clean,
                        'type': 'given_names',
                        'confidence': 0.9,
                        'source': 'mrz'
                    })

    return candidates

def calculate_name_confidence(name, name_type, full_text):
    """Calculate confidence score for a name candidate"""
    confidence = 0.5  # Base confidence

    # Length-based scoring
    if 3 <= len(name) <= 20:
        confidence += 0.2
    elif 2 <= len(name) <= 30:
        confidence += 0.1

    # Word count scoring
    words = name.split()
    if name_type == 'surname' and len(words) <= 2:
        confidence += 0.2
    elif name_type == 'given_names' and 1 <= len(words) <= 3:
        confidence += 0.2

    # Character composition
    if re.match(r'^[A-Za-z\s\'-]+$', name):
        confidence += 0.2

    # Context clues in surrounding text
    name_lower = name.lower()
    context_words = ['surname', 'given', 'first', 'name', 'names']
    for word in context_words:
        if word in full_text.lower():
            confidence += 0.1
            break

    # Penalize if it looks like OCR garbage
    if any(garbage in name_lower for garbage in ['fem', 'san', 'xxx']):
        confidence -= 0.3

    return min(1.0, max(0.0, confidence))

def select_best_name(candidates, name_type):
    """Select the best name from candidates based on confidence and validation"""
    if not candidates:
        return None

    # Filter by type
    type_candidates = [c for c in candidates if c['type'] == name_type or c['type'] == 'general']

    if not type_candidates:
        return None

    # Sort by confidence and pick the best
    type_candidates.sort(key=lambda x: x['confidence'], reverse=True)

    # Additional validation for the top candidate
    best = type_candidates[0]
    if best['confidence'] < 0.3:  # Too low confidence
        return None

    return best['text']

def validate_date(date_str):
    """Validate if date string is reasonable"""
    if not date_str:
        return False

    # Extract day, month, year
    parts = re.split(r'[/-]', date_str)
    if len(parts) != 3:
        return False

    try:
        day, month, year = map(int, parts)
        # Basic validation
        if not (1 <= month <= 12 and 1 <= day <= 31 and 1900 <= year <= 2030):
            return False
        # Additional checks for invalid dates like 40th month
        if month > 12 or day > 31:
            return False
        return True
    except ValueError:
        return False

def extract_from_mrz(text, existing_data):
    """Extract information from Machine Readable Zone (MRZ)"""
    data = existing_data.copy()

    # Pattern for first MRZ line: P<COUNTRY<SURNAME<<GIVEN_NAMES<<<
    mrz1_pattern = r'P<([A-Z]{3})<([A-Z<]+)<<([A-Z<]*)'
    mrz1_match = re.search(mrz1_pattern, text)

    if mrz1_match:
        country_code = mrz1_match.group(1)
        surname = mrz1_match.group(2).replace('<', '')
        given_names = mrz1_match.group(3).replace('<', ' ').strip()

        # Map common country codes
        country_mapping = {
            'IND': 'Indian', 'USA': 'American', 'GBR': 'British',
            'CAN': 'Canadian', 'AUS': 'Australian', 'DEU': 'German',
            'FRA': 'French', 'ITA': 'Italian', 'ESP': 'Spanish'
        }

        if not data.get('nationality'):
            data['nationality'] = country_mapping.get(country_code, country_code)
        if not data.get('surname') and surname:
            data['surname'] = clean_text(surname, "name")
        if not data.get('given_names') and given_names:
            data['given_names'] = clean_text(given_names, "name")

    # Pattern for second MRZ line: PASSPORT<COUNTRY<DOB<SEX<EXPIRY<
    mrz2_pattern = r'([A-Z][0-9]{7,8})<([A-Z]{3})<([0-9]{6})<([MF])<([0-9]{6})'
    mrz2_match = re.search(mrz2_pattern, text)

    if mrz2_match:
        passport_num = mrz2_match.group(1)
        dob_yymmdd = mrz2_match.group(3)
        sex = mrz2_match.group(4)

        if not data.get('passport_number'):
            data['passport_number'] = passport_num

        if not data.get('dob') and len(dob_yymmdd) == 6:
            yy, mm, dd = dob_yymmdd[:2], dob_yymmdd[2:4], dob_yymmdd[4:6]
            # Convert 2-digit year to 4-digit (assumes 19xx for 50-99, 20xx for 00-49)
            year = f"19{yy}" if int(yy) >= 50 else f"20{yy}"
            formatted_date = f"{dd}/{mm}/{year}"
            if validate_date(formatted_date):
                data['dob'] = formatted_date

        if not data.get('sex'):
            data['sex'] = sex

    return data

def extract_passport_data(text):
    """Extract passport data using enhanced name extraction"""
    data = {}

    try:
        # 1. Enhanced Name Extraction
        name_candidates = extract_name_candidates(text)

        # Select best surname and given names
        surname = select_best_name(name_candidates, 'surname')
        given_names = select_best_name(name_candidates, 'given_names')

        if surname:
            data['surname'] = surname
        if given_names:
            data['given_names'] = given_names


        # 2. Passport Number - flexible patterns
        passport_patterns = [
            r'\b([A-Z][0-9]{7,8})\b',  # Standard: Letter + 7-8 digits
            r'(?:Passport\s*(?:No|Number)[\s.:]*)?([A-Z][0-9]{7,8})',
            r'(?:IND|USA|GBR|CAN|AUS)\s+([A-Z][0-9]{7,8})',  # Country code + passport
        ]

        for pattern in passport_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                passport_num = clean_text(match.group(1), "passport_number")
                if len(passport_num) >= 8:  # Ensure minimum length
                    data['passport_number'] = passport_num
                    break

        # 3. Nationality - improved patterns and validation
        nationality_patterns = [
            r'(?:Nationality|Citizen(?:ship)?|Country)[\s:]*([A-Z][A-Z\s]+?)(?:\s*(?:\n|Sex|Date|Place))',
            r'\b(Indian|American|British|Canadian|Australian|German|French|Italian|Spanish|Chinese|Japanese|Korean|Brazilian|Mexican|Russian)\b',
        ]

        for pattern in nationality_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                nationality = clean_text(match.group(1), "name")
                # Validate nationality (must be reasonable length and not a common OCR error)
                if (nationality and len(nationality) > 3 and
                    nationality.lower() not in ['fem', 'feu', 'san', 'the', 'and', 'date', 'sex']):
                    data['nationality'] = nationality
                    break

        # Additional nationality fix: if we got gibberish, try to find "INDIAN" specifically
        if not data.get('nationality') or len(data.get('nationality', '')) < 4:
            indian_match = re.search(r'[IF]NDIAN|IND[IF]AN|[IF]ND[IF]AN', text, re.IGNORECASE)
            if indian_match:
                data['nationality'] = 'Indian'

        # 4. Sex - contextual patterns
        sex_patterns = [
            r'(?:Sex|Gender)[\s:]*([MF]|Male|Female)',
            r'\b([MF])\b(?=.*\d{1,2}[/-]\d{1,2}[/-]\d{4})',  # M/F near a date
            r'(?:Indian|American|British)\s+([MF])\s+\d',  # Nationality + Sex + date pattern
        ]

        for pattern in sex_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                sex_value = match.group(1).upper()
                data['sex'] = 'M' if sex_value in ['M', 'MALE'] else 'F'
                break

        # 5. Date of Birth - with validation
        dob_patterns = [
            r'(?:Date\s*of\s*Birth|DOB|Born)[\s:]*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'[MF]\s+(\d{1,2}[/-]\d{1,2}[/-]\d{4})',  # After sex field
            r'\b(\d{1,2}[/-]\d{1,2}[/-](?:19|20)\d{2})\b',  # Date with 4-digit year
        ]

        for pattern in dob_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                date_candidate = clean_text(match.group(1), "date")
                if validate_date(date_candidate):
                    data['dob'] = date_candidate
                    break
            if data.get('dob'):
                break

        # 6. MRZ fallback for any missing fields
        data = extract_from_mrz(text, data)

    except Exception as e:
        data['error'] = f"Extraction error: {str(e)}"

    return data

def process_passport(image):
    """Process passport image with multiple OCR strategies"""
    if image is None:
        return {}, ""

    try:
        all_texts = []

        # Strategy 1: Standard OCR
        text1 = pytesseract.image_to_string(image, lang='eng')
        all_texts.append(("STANDARD", text1))

        # Strategy 2: Multiple preprocessing methods
        for method in ["standard", "sharp", "threshold"]:
            enhanced_img = preprocess_image(image, method)
            enhanced_pil = Image.fromarray(enhanced_img)
            text = pytesseract.image_to_string(enhanced_pil, lang='eng',
                                            config='--oem 3 --psm 6')
            all_texts.append((f"ENHANCED_{method.upper()}", text))

        # Strategy 3: MRZ-focused extraction (bottom region)
        img_array = np.array(image)
        h, w = img_array.shape[:2]
        mrz_region = img_array[int(h*0.75):, :]
        mrz_img = Image.fromarray(mrz_region)
        text_mrz = pytesseract.image_to_string(mrz_img, lang='eng',
                                            config='--psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ<')
        all_texts.append(("MRZ", text_mrz))

        # Strategy 4: Try different PSM modes for problematic text
        for psm in [3, 6, 7, 8]:
            text_psm = pytesseract.image_to_string(image, lang='eng',
                                                config=f'--oem 3 --psm {psm}')
            all_texts.append((f"PSM_{psm}", text_psm))

        # Combine all text sources
        combined_sections = []
        for label, text in all_texts:
            if text.strip():
                combined_sections.append(f"=== {label} ===\n{text}")

        combined_text = "\n\n".join(combined_sections)

        # Extract data from combined text
        extracted_data = extract_passport_data(combined_text)

        return extracted_data, combined_text

    except Exception as e:
        return {"error": f"Processing failed: {str(e)}"}, ""

# Gradio Interface
with gr.Blocks(title="Enhanced Passport OCR") as demo:
    gr.Markdown("### 🛂 Enhanced Passport OCR with Improved Name Extraction")
    gr.Markdown("Advanced pattern matching with multiple validation strategies and OCR error correction.")

    with gr.Row():
        inp = gr.Image(type="pil", label="Upload Passport Image")

    with gr.Row():
        with gr.Column():
            extracted = gr.JSON(label="Extracted Data")
        with gr.Column():
            raw_text = gr.Textbox(label="OCR Text (All Sources)", lines=15)

    inp.change(fn=process_passport, inputs=inp, outputs=[extracted, raw_text])

    gr.Markdown("### Enhanced Features:")
    gr.Markdown("""
    - **Advanced Name Extraction**: Multiple strategies including context-based, MRZ, and line analysis
    - **OCR Error Correction**: Fixes common character substitutions (0→O, 1→I, etc.)
    - **Confidence Scoring**: Rates name candidates and selects the most reliable
    - **Smart Validation**: Enhanced filtering of OCR artifacts and false positives
    - **Robust Pattern Matching**: Handles various passport formats and layouts
    - **Multiple Fallback Methods**: Uses different OCR approaches when primary methods fail
    """)

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b1a521c5e4a9ecb80.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
